In [ ]:
import numpy as np
import open3d as o3d
from sklearn.cluster import DBSCAN
from scipy.spatial import ConvexHull

def adaptive_dbscan(points, colors):
    # Estimate eps based on average nearest neighbor distance
    pcd_temp = o3d.geometry.PointCloud()
    pcd_temp.points = o3d.utility.Vector3dVector(points)
    distances = pcd_temp.compute_nearest_neighbor_distance()
    eps = np.mean(distances) * 10  # You can adjust this multiplier

    # Estimate min_samples based on point cloud density
    min_samples = max(30, int(len(points) * 0.001))  # At least 10 points, or 0.1% of total points
    print("eps:", eps, "min_samples:", min_samples)
    # Perform DBSCAN clustering
    dbscan = DBSCAN(eps=eps, min_samples=min_samples, n_jobs=-1)
    labels = dbscan.fit_predict(points)

    return labels

def analyze_cluster(points, colors):
    # Compute convex hull for shape analysis
    hull = ConvexHull(points)
    volume = hull.volume
    surface_area = hull.area
    compactness = (surface_area ** 3) / (36 * np.pi * volume ** 2)

    # Color analysis
    avg_color = np.mean(colors, axis=0)
    color_std = np.std(colors, axis=0)

    return volume, compactness, avg_color, color_std

def is_valid_cluster(points, colors, volume, compactness, avg_color, color_std):
    # Adjust these thresholds based on your specific use case
    min_points = 30
    max_volume = 10**-2  # Adjust based on your scene scale
    max_compactness = 10  # Lower values indicate more compact shapes
    color_threshold = 0.2  # Adjust based on color variation in your objects of interest
    print("points:", len(points), "volume:", volume, "compactness:", compactness)
    print(len(points) < min_points, volume > max_volume, compactness > max_compactness)
    if len(points) < min_points or volume > max_volume or compactness > max_compactness:
        return False
    print("passed")
    # Check color consistency
    print("color_std:", color_std)
    if np.any(color_std > color_threshold):
        return False

    # Add more specific color checks if needed, e.g., for pistil color
    # if not is_pistil_color(avg_color):
    #     return False

    return True

def improved_test_dbscan(pcd_final):
    points = np.asarray(pcd_final.points)
    colors = np.asarray(pcd_final.colors)
    print("points:", points.shape)
    # Perform adaptive DBSCAN clustering
    labels = adaptive_dbscan(points, colors)
    unique_labels = set(labels)
    print("labels:", labels)
    print("Number of unique labels:", len(unique_labels))
    max_label = labels.max()
    all_segments = o3d.geometry.PointCloud()
    bounding_boxes = []

    for i in range(max_label + 1):
        cluster_points = points[labels == i]
        cluster_colors = colors[labels == i]

        volume, compactness, avg_color, color_std = analyze_cluster(cluster_points, cluster_colors)
        print("i:", i)
        if is_valid_cluster(cluster_points, cluster_colors, volume, compactness, avg_color, color_std):
            new_pc = o3d.geometry.PointCloud()
            new_pc.points = o3d.utility.Vector3dVector(cluster_points)
            new_pc.colors = o3d.utility.Vector3dVector(cluster_colors)
            print("isvalid:", i)
            bb = new_pc.get_oriented_bounding_box()
            bounding_boxes.append(bb)

            all_segments += new_pc

    visualizer(all_segments)
    # o3d.io.write_point_cloud(f"temp/{folder}/pcd_DB.ply", all_segments)

    return bounding_boxes, all_segments

In [ ]:
# def rotate_point_cloud_90_x(vis, point_cloud):

#     zoom = 0.7
#     front = [0.22650835232871624, 0.96909623379403964, 0.097705966918803414]
#     lookat = [ 0.47142450361015009, -6.2676496329405751e-05, 0.92503262586022961 ]  # Replace with your center of mass calculation
#     up = [-0.97258606651013824, 0.21961578825386974, 0.076454226697864364]
#     for deg in range(0, 361, 90):
#         vis.clear_geometries()
        
#         transformed_cloud = point_cloud

#         theta = math.radians(90)
        
#         # Define rotation matrix around x-axis
#         rotation_matrix = np.array([
#             [1,             0,              0, 0],
#             [0, np.cos(theta), -np.sin(theta), 0],
#             [0, np.sin(theta),  np.cos(theta), 0],
#             [0,             0,              0, 1]
#         ])

#         # y-axis
#         # rotation_matrix = np.array([
#         #     [np.cos(theta),  0, np.sin(theta), 0],
#         #     [0,              1,             0, 0],
#         #     [-np.sin(theta), 0, np.cos(theta), 0],
#         #     [0,              0,             0, 1]
#         # ])
        
#         # z-axos
#         # rotation_matrix = np.array([
#         #     [np.cos(theta), -np.sin(theta), 0, 0],
#         #     [np.sin(theta),  np.cos(theta), 0, 0],
#         #     [0,              0,             1, 0],
#         #     [0,              0,             0, 1]
#         # ])

#         # Apply transformation to the point cloud
#         transformed_cloud.transform(rotation_matrix)
#         scale_factor = 1.5  # Example scale factor (adjust as needed)
#         transformed_cloud.scale(scale_factor, center=transformed_cloud.get_center())
#         vis.add_geometry(transformed_cloud)
#         # vis.get_view_control().scale(2)
#         vis.update_geometry(transformed_cloud)
#         vis.update_renderer()
#         vis.capture_screen_image(f"screenshot_axis_{deg}.png", do_render=True)

# vis = o3d.visualization.Visualizer()
# vis.create_window()
# vis.run()
# rotate_point_cloud_90_x(vis, point_cloud)
# vis.destroy_window()


In [ ]:
# data = o3d.io.read_pinhole_camera_parameters("2.json")
# view_control = o3d.visualization.ViewControl()
# cam_to_world = view_control.convert_from_pinhole_camera_parameters(data)
# pcd.transform(cam_to_world)
# # visualizer(pcd)
# o3d.visualization.draw_geometries([pcd])

In [ ]:
# vis = o3d.visualization.Visualizer()
# vis.create_window()
# vis.add_geometry(pcd)
# ctr = vis.get_view_control()
# print("Field of view (before changing) %.2f" % ctr.get_field_of_view())
# # ctr.set_front([ 0.22650835232871624, 0.96909623379403964, 0.097705966918803414 ])
# # ctr.change_field_of_view(step=-10)
# print("Field of view (after changing) %.2f" % ctr.get_field_of_view())
# vis.run()
# vis.destroy_window()

In [ ]:
# plane ransac
sphere = pyrsc.Plane()
# s = 3
equation, best_inliers_plane = sphere.fit(np.asarray(pcd.points), thresh=0.1, maxIteration=40)

inlier_cloud_sphere = pcd.select_by_index(best_inliers_plane)
outlier_cloud_sphere = pcd.select_by_index(best_inliers_plane, invert=True)

# outlier_cloud_sphere.paint_uniform_color([1, 0, 0])
inlier_cloud_sphere.paint_uniform_color([0, 1, 0])

visualizer([inlier_cloud_sphere, outlier_cloud_sphere])

In [ ]:
pcd = outlier_cloud_sphere

In [ ]:
mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=1, origin=pcd.get_center())

visualizer([pcd, mesh_frame])

In [ ]:
# Ax+By+Cy+D
A, B, C, D = equation

plane_normal = np.array([A, B, C])

# v1 and v2 are new x, y axes
v1 = np.array([1, 0, -A/C])
v1 -= np.dot(v1, plane_normal) * plane_normal / np.dot(plane_normal, plane_normal)
v1 /= np.linalg.norm(v1)
v2 = np.cross(plane_normal, v1)

R = np.stack((v1, v2, plane_normal))

rotated_points = np.dot(pcd.points, R.T)

point_cloud = o3d.geometry.PointCloud()
point_cloud.points = o3d.utility.Vector3dVector(rotated_points)
point_cloud.colors = o3d.utility.Vector3dVector(pcd.colors)

mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=1, origin=point_cloud.get_center())

visualizer([point_cloud, mesh_frame])


In [ ]:
pcd.points = o3d.utility.Vector3dVector(pcd.points - np.mean(np.asarray(pcd.points), axis=0))

In [ ]:
center_of_mass = np.mean(np.asarray(pcd.points), axis=0)
    
o3d.visualization.draw_geometries([pcd],
                            zoom=0.5,
                            front=[ 1, 0, 0 ],
                            up = [ 0, -1, 0 ],
                            lookat=[0, 0, 0])

In [ ]:
pcd2 = o3d.geometry.PointCloud()
pcd2.points = o3d.utility.Vector3dVector(pcd.points)
pcd2.colors = o3d.utility.Vector3dVector(pcd.colors)

vis = o3d.visualization.Visualizer()
vis.create_window()
scale_factor = 10
pcd2.scale(scale_factor, center=pcd2.get_center())
vis.add_geometry(pcd2)
ctr = vis.get_view_control()
front = [1, 0, 0]
ctr.set_front(front)
ctr.set_up([1, 0, 0])
ctr.set_zoom(1000)

vis.add_geometry(pcd2)

# for i in range(0, 361, 90):
#     vis.clear_geometries()
#     pcd2 = pcd
#     pcd2.scale(scale_factor, center=pcd2.get_center())
#     R = pcd2.get_rotation_matrix_from_xyz((0, 0, 90))
#     pcd2=pcd2.rotate(R, center= np.mean(np.asarray(pcd2.points), axis=0))
#     vis.add_geometry(pcd2)
#     vis.update_geometry(pcd2)
#     vis.poll_events()
#     vis.update_renderer()
#     vis.capture_screen_image(f"screenshot_axis1_{i}.png", do_render=False)

# vis.destroy_window()


In [ ]:
o3d.visualization.draw_geometries_with_editing([point_cloud])

In [ ]:
rf = Roboflow(api_key="p0DKMSH5Ym3FH9zh7ZLa")
project = rf.workspace().project("sweet-pepper-detection")
model = project.version(1).model

data = model.predict("numpy_image_2.jpg", confidence=10, overlap=30).json()

# visualize your prediction
# model.predict("numpy_image_2.jpg", confidence=10, overlap=30).save("predictions/prediction2.jpg")

In [ ]:
import open3d as o3d
import numpy as np

# 2D bounding box coordinates
bounding_box = {
    'x': data['predictions'][0]['x'],
    'y': data['predictions'][0]['y'],
    'width': data['predictions'][0]['width'],
    'height': data['predictions'][0]['height'],
}

# center

# Image dimensions (to scale 2D to 3D)
image_width = data['image']['width']
image_height = data['image']['height']

# Calculate 3D coordinates (assuming z-coordinate is arbitrary)
x_min = bounding_box['x'] - bounding_box['width']/2
y_min = bounding_box['y'] - bounding_box['height']/2
x_max = bounding_box['x']  + bounding_box['width']/2
y_max = bounding_box['y'] + bounding_box['height']/2

In [ ]:
max_vals = np.min(np.asarray(pcd.points), axis=0)
min_vals = np.max(np.asarray(pcd.points), axis=0)

In [ ]:
# min = [-0.67169776 -0.92427411 -0.84997463]
# [0.70820196 1.07102211 0.94163357]

x_new = (x_min/1920) * np.abs(max_vals[0]) + np.abs(min_vals[0]) - np.abs(min_vals[0])
y_new = (y_min/1920) * np.abs(max_vals[1]) + np.abs(min_vals[1]) - np.abs(min_vals[1])
xx_new = (x_max/1920) * np.abs(max_vals[0]) + np.abs(min_vals[0]) - np.abs(min_vals[0])
yy_new = (y_max/1920) * np.abs(max_vals[1]) + np.abs(min_vals[1]) - np.abs(min_vals[1])

print(x_new, y_new, xx_new, yy_new)

In [ ]:
# Define the bounding box in 3D space
bbox_min = np.array([[x_new, y_new, 0.0]])
bbox_max = np.array([xx_new, yy_new, 0.0])
bbox_color = (1, 0, 0)  # Red color

# Create a bounding box using Open3D
bbox = o3d.geometry.AxisAlignedBoundingBox(min_bound=bbox_min, max_bound=bbox_max)
bbox_lines = o3d.geometry.LineSet.create_from_axis_aligned_bounding_box(bbox)
bbox_lines.paint_uniform_color(bbox_color)

In [ ]:
# Visualize cloud and edit
vis = o3d.visualization.VisualizerWithEditing()
vis.create_window()
vis.add_geometry(pcd)
vis.run()
vis.destroy_window()
print(vis.get_picked_points())

In [ ]:
vertices = []
for i in np.arange(-2, 2, 0.01):
    vertices.append([x_new, i, i])
    vertices.append([i, y_new, i])
    vertices.append([x_new, y_new, i])
    vertices.append([xx_new, yy_new, i])
    vertices.append([x_new, i, y_new])
    vertices.append([xx_new, i, yy_new])
    vertices.append([i, x_new, y_new])
    vertices.append([i, xx_new, yy_new])

# vertices = [[-2, -2, -2], [2, 2, 2], np.mean(np.asarray(pcd.points), axis=0)]
vertices = np.array(vertices)

In [ ]:
# Create a point cloud from numpy array
point_cloud = o3d.geometry.PointCloud()
point_cloud.points = o3d.utility.Vector3dVector(vertices)
point_cloud.paint_uniform_color(bbox_color)

In [ ]:
# plane ransac
sphere = pyrsc.Plane()
# s = 3
equation, best_inliers_plane = sphere.fit(np.asarray(pcd.points), thresh=0.2, maxIteration=40)

inlier_cloud_sphere = pcd.select_by_index(best_inliers_plane)
outlier_cloud_sphere = pcd.select_by_index(best_inliers_plane, invert=True)

# outlier_cloud_sphere.paint_uniform_color([1, 0, 0])
inlier_cloud_sphere.paint_uniform_color([0, 1, 0])

visualizer([inlier_cloud_sphere, outlier_cloud_sphere])

In [ ]:
# Graph Paper Method ineffeient code

# for dx in range(-resolution, resolution, 1):
#     for dz in range(-resolution, resolution, 1):

#         if 0 <= x_c + dx < shape1 and 0 <= z_c + dz < shape2: #If it is in bounds with the image shapes
#             if not check_grid[x_c + dx, z_c + dz]:
#                 check_grid[x_c + dx, z_c + dz] = 1
#                 img[x_c + dx, z_c + dz] = col
